## Writing Functions

The goal of this exercise is to replicate the process from the slides, but with multivariate data. You should think of this as a practise of two things: 

1. Performing linear algebra in R. This is straight forward, but takes some practise. Especially worth noting how to use vectors and scalars together, as you see how to modify the functions we originally wrote for scalars, such that they work for vectors.

2. Writing small functions and testing them. Make sure each part works before trying to combine them into a whole!

In [1]:
library(tidyverse)

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats


In [2]:
# Write a function that generates data for regressions, 

# y should be generated according to: 
# y <- X %*% beta + eps
# X should be several columns of independent random normals (aka a multivariate random 
# normal matrix with diagonal covariance)
# eps should be such that:
# eps <- rnorm(1, 0, sd)

# beta should be a vector
# params should be a DATAFRAME/TIBBLE with two columns: "mean" and "sd" to generate the features (X) 
# sd should be a scalar: the standard deviation of the normally distributed error term in the DGP

# hint: use cbind to bind vectors into the columns of a matrix or the library MASS with the function mvrnorm

generate_data <- function(N, beta, params, sd) {
      means <- params$mean
  sds <- params$sd
  x <- rep(1, N)
  
for (i in 1:length(means)) {
    
    xi <- rnorm(N, means[i], sds[i]) 
    x <- cbind(x,xi)
  
    }
       
       eps <- rnorm(N, 0, sd)
       y <- x%*%beta + eps
       list(x = x, y = y)
}


calc_coef <- function(y,x) {
  coef <- solve(t(x)%*%x, t(x)%*%y)
    coef
}


calc_se <- function(y, x, coef) {
      
      eps <- y - x %*% coef
      sigma2 <- (t(eps) %*% eps) / (nrow(x) - ncol(x))
      se <- c(sigma2) * solve(t(X) %*% X)
    se
      
}

run_regression <- function(y, x) {
    coef <- calc_coef(y, x)
    se <- calc_se(y, x, coef)
    list(coef=coef, se=se)
}

#Works until here

eval_model <- function(coef, se, beta, conf = 1.96) {
    up <- coef + se*conf
    down <- coef - se*conf
    beta > down & beta < up
}

  simulate <- function(N, beta, sd) {
       d <- generate_data(N, beta, sd)
       m <- run_regression(d$y, d$x)
       eval_model(m$coef, m$se, beta)
   }
   avg_simulations <- function(M, N, beta, sd) {
       inside <- sapply(1:M, function (x) {
           simulate(N, beta, sd)
})
       sum(inside) / M
   }

In [326]:
# Write other helper functions, as in the slides, and eventually a function "avg_simulations" that takes
# the same parameters as generate_data, plus an "M" parameter that controls how many simulations are run

## Tests

Make sure the following assertions pass:

In [4]:
a <- avg_simulations(1000, 
                20, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(round(a, 1) == .9)

ERROR: Error in avg_simulations(1000, 20, c(1, 2, 1), data.frame(list(mean = c(0, : unused argument (0.5)


In [ ]:
a <- avg_simulations(1000, 
                500, 
                c(1,2,1), 
                data.frame(list(mean=c(0,0,0), sd=c(.2, .5, .3))), 
                .5)

stopifnot(a > .92)